# How to Use st_hsdatalog Package - \[Data Format Conversion Features\]
---
<br>
<p>In this notebook, we will explore how to use the `st_hsdatalog` package, specifically focusing on data format conversion features. This guide will walk you through the process of importing necessary modules, initializing objects, and converting data into various formats.</p>

## Important Notice
Please ensure to check the names of the exported files to avoid overwriting any existing files. This is crucial when running multiple conversions or working with different datasets.

In [ ]:
from st_hsdatalog.HSD.HSDatalog import HSDatalog

<span style="font-style: italic; color:#909090;"><span style="color:cyan;">*[Notebook utils] </span> -> Update this string to select the right acquisition folder</span>

In [ ]:
#acquisition_folder = "path/to/your/acquisition_folder"
acquisition_folder = "20240712_15_18_51"

## Initialize HSDatalog Object and Validate Acquisition Folder
We will initialize the `HSDatalog` object and validate the acquisition folder. This step ensures that the folder contains the necessary data and is in the correct format.
<span style="color:#909090;">[FP-SNS-DATALOG1 and FP-SNS-DATALOG2 acquisition formats are supported]</span>

In [ ]:
hsd = HSDatalog()
hsd_version = hsd.validate_hsd_folder(acquisition_folder)
print(f"HSD Version: {hsd_version}\n")

hsd_instance = hsd.create_hsd(acquisition_folder=acquisition_folder)

## Extract DataFrames from HSDatalog Instance
The created `HSDatalog` instance will be used to extract Pandas dataframes from an HSDatalog acquisition folder. Ensure to change the `sensor_name` string to a valid sensor name available in your acquisition folder.

In [ ]:
sensor_name = "ism330dhcx_acc"
try:
    sensor =  hsd.get_sensor(hsd_instance, sensor_name)
    dataframe = hsd.get_dataframe(hsd_instance, sensor)
    print(dataframe)
except Exception as e:
    print(f"Error: No [{sensor_name}] sensor available in your selected acquisition folder. Please check the sensor name and try again.")

# Data Conversion
---
<p>Using the `HSDatalog` module, it is possible to convert the data of an acquisition into various formats. The supported formats include:</p>
<ul>
<li> CSV, TSV, Apache Parquet, TXT (also split into different folders by tags)</li>
<li> Nanoedge format</li>
<li> ST UNICO format (with variants)</li>
<li> WAV</li>
</ul>

First, create an output folder to host the converted files.

In [ ]:
import os

output_folder = ".\\HSD_Exported_data_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

## CSV, TSV, Apache Parquet and TXT Conversion
In this section, we will demonstrate how to convert data into CSV, TSV, and Apache Parquet formats. We will also show how to handle TXT or CSV conversion by tags.

### CSV and TSV Conversion Examples
Below are examples of how to convert data to CSV and TSV formats. Note that the `convert_dat_to_xsv` function is used for these conversions.

In [ ]:
from st_hsdatalog.HSD_utils.exceptions import MissingTagsException

exported_sensor_path = os.path.join(output_folder, sensor_name)

print(f"To CSV conversion started...")
hsd.convert_dat_to_xsv(hsd_instance, sensor, start_time=0, end_time=-1, labeled=False, raw_data=False, output_folder=output_folder, file_format="CSV")
print(f"To CSV conversion completed.")
print(f"To TSV conversion started...")
try:
    hsd.convert_dat_to_xsv(hsd_instance, sensor, start_time=0, end_time=-1, labeled=True, raw_data=True, output_folder=output_folder, file_format="TSV")
    print(f"To TSV conversion completed.")
except MissingTagsException as e:
    print(f"Error: No tags are present in your input acquisition folder. Please use labeled=False parameter or select a labeled acquisition.")
print(f"To Apache Parquet conversion started...")
hsd.convert_dat_to_xsv(hsd_instance, sensor, start_time=0, end_time=-1, labeled=False, raw_data=False, output_folder=output_folder, file_format="PARQUET")
print(f"To Apache Parquet conversion completed.")

### TXT, CSV Conversion by Tags
When an acquisition has been recorded with labels, it is possible to convert .dat files considering those labels. Below are examples of how to convert data to TXT format, filtered by tags.

In [ ]:
print(f"To TXT Conversion filtered by tags started...")
hsd.convert_dat_to_txt_by_tags(hsd_instance, sensor, start_time=0, end_time=-1, output_folder=output_folder, with_untagged = False, out_format="TXT")
print(f"To TXT Conversion filtered by tags completed.")
print(f"To TXT Conversion filtered by tags (with untagged folder) started...")
hsd.convert_dat_to_txt_by_tags(hsd_instance, sensor, start_time=0, end_time=-1, output_folder=output_folder, with_untagged = True, out_format="CSV")
print(f"To TXT Conversion filtered by tags (with untagged folder) completed.")

## Nanoedge Format Conversion
Below an example of how to convert .dat files to Nanoedge format.

In [ ]:
print(f"To Nanoedge format conversion started...")
hsd.convert_dat_to_nanoedge(hsd_instance, sensor, signal_length=1000, signal_increment=500, start_time=0, end_time=-1, raw_data=False, output_folder=output_folder)
print(f"To Nanoedge format conversion completed.")

## ST UNICO Format Conversion
Below are examples of how to convert .dat files to ST UNICO format for both unlabeled and labeled acquisitions.

### Unlabeled Acquisition
Even if present, the tag data in the acquisition is not used.

In [ ]:
print(f"To Unico format conversion started...")
hsd.convert_dat_to_unico(hsd_instance, [sensor], start_time=0, end_time=-1, use_datalog_tags=False, output_folder=output_folder, out_format="TXT")
print(f"To Unico format conversion completed.")

### Labeled Acquisition
If available, the tag data are read and used to split the converted files.

In [ ]:
print(f"To Unico format conversion started...")
hsd.convert_dat_to_unico(hsd_instance, [sensor], start_time=0, end_time=-1, use_datalog_tags=True, output_folder=output_folder, out_format="CSV")
print(f"To Unico format conversion completed.")

### Aggregated Data
Aggregated data can be saved in different files (one per tag group) or as a single file.

- ##### Single File Aggregated Data
A single aggregated file is generated as output.

In [ ]:
print(f"To Unico (\"single_file\" Aggregated) format conversion started...")
hsd.convert_dat_to_unico_aggregated(hsd_instance, aggregation="single_file", start_time=0, end_time=-1, use_datalog_tags=False, output_folder=output_folder, with_untagged = True, out_format="CSV")
print(f"To Unico (\"single_file\" Aggregated) format conversion completed.")

- ##### Split Per Tags Aggregated Data
Aggregated data are saved in different files (one per tag group, organized in one folder per tag class).

In [ ]:
print(f"To Unico (\"split_per_tags\" Aggregated) format conversion started...")
hsd.convert_dat_to_unico_aggregated(hsd_instance, aggregation="split_per_tags", start_time=0, end_time=-1, use_datalog_tags=True, output_folder=output_folder, with_untagged = False, out_format="CSV")
print(f"To Unico (\"split_per_tags\" Aggregated) format conversion completed.")
print(f"To Unico (\"split_per_tags\" Aggregated, with untagged folder) format conversion started...")
hsd.convert_dat_to_unico_aggregated(hsd_instance, aggregation="split_per_tags", start_time=0, end_time=-1, use_datalog_tags=True, output_folder=output_folder, with_untagged = True, out_format="TXT")
print(f"To Unico (\"split_per_tags\" Aggregated), with untagged folder format conversion completed.")

## WAV Conversion
The WAV format is used for audio data. Below is an example of how to convert data to WAV format.

In [ ]:
sensor_name = "imp23absu_mic"
try:
    sensor =  hsd.get_sensor(hsd_instance, sensor_name)
except Exception as e:
    print(f"Error: No [{sensor_name}] sensor available in your selected acquisition folder. Please check the sensor name and try again.")
print(f"To Wav conversion started...")
hsd.convert_dat_to_wav(hsd_instance, sensor, start_time=0, end_time=-1, output_folder=output_folder)
print(f"To Wav conversion completed.")